In [1]:
import pandas as pd

In [2]:
buy=pd.read_csv('purchase_history.csv')
item=pd.read_csv('item_to_id.csv')

In [15]:
buy=buy.set_index('user_id')

In [16]:
buy.head()

,id
user_id,
222087,"27,26"
1343649,"6,47,17"
404134,"18,12,23,22,27,43,38,20,35,1"
1110200,"9,23,2,20,26,47,37"
224107,"31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4..."


In [17]:
item.head()

,Item_name,Item_id
0,coffee,43
1,tea,23
2,juice,38
3,soda,9
4,sandwich loaves,39


In [18]:
item_dict=dict(zip(item['Item_id'],item['Item_name']))

# custermor buy most items

In [19]:
buy['total']=buy['id'].apply(lambda x:len(x.split(',')))

In [20]:
buy.total.idxmax()

388149

In [21]:
buy.loc[388149]   #duplicate user_id

,id,total
user_id,,
388149,"18,38,36,7,1,10,45,26,39,5,44,40,32,25,9,21,28...",27
388149,"34,23,13,18,37,21,16",7


In [22]:
new_buy=buy.groupby(['user_id']).total.agg('sum').reset_index()
new_buy.loc[new_buy.total.idxmax()]

user_id    269335
total          72
Name: 4445, dtype: int64

In [23]:
print('custom {}, buy most items'.format(new_buy.user_id[new_buy.total.idxmax()]))

custom 269335, buy most items


# cusmter buy most for each item

In [24]:
x=buy.loc[388149]

In [26]:
y=x['id'].str.split(',').sum()  #concatenate two list together
y

['18',
 '38',
 '36',
 '7',
 '1',
 '10',
 '45',
 '26',
 '39',
 '5',
 '44',
 '40',
 '32',
 '25',
 '9',
 '21',
 '28',
 '34',
 '13',
 '41',
 '27',
 '19',
 '33',
 '35',
 '4',
 '43',
 '48',
 '34',
 '23',
 '13',
 '18',
 '37',
 '21',
 '16']

In [28]:
from collections import Counter

In [29]:
def item_counts(df):
    y=df['id'].str.split(',').sum()
    return(pd.Series(Counter(y)))

In [30]:
buy_matrix=buy.groupby(['user_id']).apply(item_counts).unstack(fill_value=0)

In [31]:
buy_matrix.head()

,1,10,11,12,13,14,15,16,17,18,...,44,45,46,47,48,5,6,7,8,9
user_id,,,,,,,,,,,,,,,,,,,,,
47,0,0,0,0,1,0,1,1,0,0,...,1,1,1,0,0,0,0,0,0,0
68,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
113,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
123,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
223,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [32]:
buy_most_matrix=pd.DataFrame(buy_matrix.idxmax()).reset_index()

In [33]:
buy_most_matrix.columns=['item_id','user_id']
buy_most_matrix.head()

,item_id,user_id
0,1,31625
1,10,618914
2,11,367872
3,12,557904
4,13,653800


In [34]:
buy_most_matrix['item_name']=buy_most_matrix['item_id'].apply(lambda x:item_dict[int(x)])

In [35]:
buy_most_matrix.head()

,item_id,user_id,item_name
0,1,31625,sugar
1,10,618914,carrots
2,11,367872,cereals
3,12,557904,shampoo
4,13,653800,bagels


In [36]:
test=buy_matrix.apply(lambda s:pd.Series([s.idxmax(),s.max()]))  #user and max counts

In [38]:
test.T.head()

,0,1
1,31625,4
10,618914,4
11,367872,3
12,557904,3
13,653800,4


# item clusting

In [118]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

ModuleNotFoundError: No module named 'mlxtend'

# market basket analysis

In [119]:
buy.head()

,id,total
user_id,,
222087,"27,26",2
1343649,"6,47,17",3
404134,"18,12,23,22,27,43,38,20,35,1",10
1110200,"9,23,2,20,26,47,37",7
224107,"31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4...",20


In [137]:
buy_matrix_new=buy.reset_index().drop(['user_id'],axis=1)

In [145]:
buy_matrix_new=buy_matrix_new.groupby(buy_matrix_new.index).apply(item_counts).unstack(fill_value=0)

In [146]:
buy_matrix_new.head()

,1,10,11,12,13,14,15,16,17,18,...,44,45,46,47,48,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1,0,0,1,1,0,0,1,0,1,...,1,0,0,1,1,1,0,1,0,0


In [206]:
def support(lst):
    n=len(lst)
    df=pd.DataFrame()
    for i in range(n):
        x=buy_matrix_new[lst[i]]
        df=pd.concat([df,x],axis=1)
    df['min']=df.min(axis=1)
    return(df['min'].sum()/buy_matrix_new.shape[0])

In [241]:
def confidence(a,b):
    return(support(a+b)/support(a))
def lift(a,b):
    return(confidence(a,b)/support(b))

In [147]:
a=[3,4,5]

In [154]:
b=[3,4]
b[0] in a

True

In [153]:
len(b)

1

In [160]:
pd.DataFrame(buy_matrix_new['11']).min(axis=1).sum()

2730

In [235]:
lift(['11'],['12'])

1.0745014618409114

In [187]:
from itertools import permutations

In [44]:
product=[]
lift_=[]
name=[]
item_list=list(map(str,item['Item_id'].unique()))
def most_lift(num):
    for j in item_list:
        if [j]!=num:
            product.append(j)
            lift_.append(lift(num,[j]))
            name.append(item_dict[int(num)])
    return(pd.DataFrame({'item':product,'lift':lift_,'name':name}).sort_values(['lift'],ascending=False))

In [282]:
most_lift(['11'])

,item,lift
19,5,2.037756
16,30,1.983969
18,31,1.969562
17,1,1.419137
45,4,1.272174
36,27,1.253222
40,7,1.202422
39,15,1.194295
13,16,1.192858
46,3,1.182198


In [3]:
item.head()

,Item_name,Item_id
0,coffee,43
1,tea,23
2,juice,38
3,soda,9
4,sandwich loaves,39


In [5]:
item.loc[item['Item_id']==11]

,Item_name,Item_id
16,cereals,11


In [9]:
item.loc[item['Item_id'].isin ([5,30,31,1])]

,Item_name,Item_id
17,flour,30
18,sugar,1
19,pasta,31
20,waffles,5


In [43]:
int('11')

11